In [1]:
# Imports
import gymnasium as gym
from typing import Callable

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import VecMonitor
import torch as th
from stable_baselines3.common.callbacks import EvalCallback

import pickle

# Import Our environment
from dev_env import tradingEng



c:\Users\osc16\miniconda3\envs\d2d\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
# Load Paths
with open("ZeroCorrFrs1Half","rb") as fp:
    paths1 = pickle.load(fp)

# Load Paths
with open("ZeroCorrFrs2Half","rb") as fp:
    paths1 = paths1 + pickle.load(fp)

with open("ZeroCorrSnd1Half","rb") as fp:
    paths2 = pickle.load(fp)

with open("ZeroCorrSnd2Half","rb") as fp:
    paths2 = paths2 + pickle.load(fp)

with open("ZeroCorrTest","rb") as fp:
    paths_ev = pickle.load(fp)


In [ ]:

## LR schedule
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func
policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam, log_std_init = 0.005)) #


In [ ]:
# Horizons

# Default
#t = start_and_release(paths1,action='small-More-Trust', obs = 'auto')
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs')
]))
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_model',
    log_path='./logs/eval_logs/',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)

model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

# 1 yr
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs', resetdate=1.0),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs',resetdate=1.0)
]))
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs',resetdate=1.0),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_model1',
    log_path='./logs/eval_logs1',
    eval_freq=252*8*1,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)

# Instantiate the agent
model = PPO("MlpPolicy", envs, batch_size = 252*2*1, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*1, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

# 20 yr
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs',resetdate=20.0),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs',resetdate=20.0)
]))
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs',resetdate=20.0),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_model20',
    log_path='./logs/eval_logs20',
    eval_freq=252*8*20,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)

model = PPO("MlpPolicy", envs, batch_size = 252*2*20, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*20, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

Using cpu device
Eval num_timesteps=20160, episode_reward=-0.10 +/- 0.03
Episode length: 1259.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 1.26e+03 |
|    mean_reward     | -0.0954  |
| time/              |          |
|    total_timesteps | 20160    |
---------------------------------
New best mean reward!
Eval num_timesteps=40320, episode_reward=-0.09 +/- 0.03
Episode length: 1259.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 1.26e+03    |
|    mean_reward          | -0.0894     |
| time/                   |             |
|    total_timesteps      | 40320       |
| train/                  |             |
|    approx_kl            | 0.002400974 |
|    clip_fraction        | 0.00796     |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.8        |
|    explained_variance   | -0.656      |
|    learning_rate        | 0.0015      |

IndexError: index 5040 is out of bounds for axis 0 with size 5040

In [ ]:
# Big observation
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'small'),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'small')
]))
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'small'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelBigObs',
    log_path='./logs/eval_logsBigObs',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)
model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 




Using cpu device
Eval num_timesteps=20160, episode_reward=-0.12 +/- 0.05
Episode length: 1259.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 1.26e+03 |
|    mean_reward     | -0.121   |
| time/              |          |
|    total_timesteps | 20160    |
---------------------------------
New best mean reward!
Eval num_timesteps=40320, episode_reward=-0.09 +/- 0.05
Episode length: 1259.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 1.26e+03     |
|    mean_reward          | -0.0907      |
| time/                   |              |
|    total_timesteps      | 40320        |
| train/                  |              |
|    approx_kl            | 0.0031787264 |
|    clip_fraction        | 0.0127       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.82        |
|    explained_variance   | -0.412       |
|    learning_rate        | 0

In [ ]:
# Big Action
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'big', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'big', obs = 'xs')
]))
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'big', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelBigAct',
    log_path='./logs/eval_logsBigAct',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)
model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 



In [ ]:
# Big 2 Big
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'big', obs = 'small'),
    lambda: tradingEng(paths2,action = 'big', obs = 'small')
]))
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'big', obs = 'small'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelBigAct',
    log_path='./logs/eval_logsBigAct',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)
model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.0015), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

In [ ]:
# Reward l2
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small', obs = 'xs', reward = 'L2'),
    lambda: tradingEng(paths2,action = 'small', obs = 'xs', reward = 'L2')
]))

ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs', reward = 'L2'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelL2',
    log_path='./logs/eval_logsL2',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)

# Instantiate the agent
policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam, log_std_init = 0.005)) #
model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 
# Save the agent
model.save("0.7basfall")

In [ ]:
# Correlations 0.7
#t = start_and_release(paths1,action='small-More-Trust', obs = 'auto')
with open("0.7Corr1Half.pkl","rb") as fp:
    paths1 = pickle.load(fp)

# Load Paths
with open("0.7Corr2Half.pkl","rb") as fp:
    paths2 = pickle.load(fp)

with open("0.7CorrTest.pkl","rb") as fp:
    paths_ev = pickle.load(fp)

envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'small', obs = 'xs')
]))

ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_model07',
    log_path='./logs/eval_logs07',
    eval_freq=252*8*5*20,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)

# Instantiate the agent
policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam, log_std_init = 0.005)) #
model = PPO("MlpPolicy", envs, batch_size = 252*4*5, learning_rate=linear_schedule(0.0015), policy_kwargs=policy_kwargs, n_steps=252*8*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=1e7, log_interval=2, callback=eval_callback) 

